In [1]:
import scanpy as sc

In [2]:
def adata_to_nmf_factor_expression(adata, cluster_label, nmf_coeffs, sample_name, add_density=True):
    """
    Convert an AnnData to a new AnnData with cluster expressions. Clusters are based on `cluster_label` 
    in `adata.obs`.  The returned AnnData has an observation for each cluster, with the 
    cluster-level expression equals to the average expression for that cluster.
    All annotations in `adata.obs` except `cluster_label` are discarded in the returned AnnData.
    
    Args:
        adata (AnnData): NMF `W` AnnData object (obs=factor, var=genes)
        cluster_label (String): field in `adata.obs` used for aggregating values
        nmf_coeffs (pd.DataFrame): NMF `H` matrix (factor by sample)
        sample_name (String): name of the sample to be mapped as labeled in `nmf_coeffs`
        scale (bool): Optional. Whether weight input single cell by # of cells in cluster. Default is True.
        add_density (bool): Optional. If True, the normalized number of cells in each cluster is 
        added to the returned AnnData as obs.cluster_density. Default is True.

    Returns:
        AnnData: aggregated single cell data

    """
    try:
        unique_labels = adata.obs[cluster_label].unique()
    except KeyError as e:
        raise ValueError("Provided label must belong to adata.obs.")
    
    new_obs = pd.DataFrame({cluster_label: unique_labels})
    adata_ret = sc.AnnData(obs=new_obs, var=adata.var, uns=adata.uns)
    X_new = np.empty((len(unique_labels), len(adata.var))).astype(object)
    
    for index, l in enumerate(unique_labels):
        X_new[index] = np.multiply(adata[adata.obs[cluster_label] == l].X, nmf_coeffs[sample_name].loc[l])
    adata_ret.X = X_new

    if add_density:
        adata_ret.obs["cluster_density"] = adata_ret.obs[cluster_label].map(
            lambda i: nmf_coeffs[sample_name][i]
        )
        adata_ret.obs['cluster_density'] /= adata_ret.obs['cluster_density'].sum()

    return adata_ret

## Load inputs

- nmf `W` loaded as ann data
- nmf `H`

In [3]:
data_path = '/gstore/home/tripaa18/MEL/melanoma-st/priya_analysis/data/'

In [4]:
adata = sc.read_h5ad(data_path+'melanoma_nmf_w_1500_norm10k.h5ad')
adata

AnnData object with n_obs × n_vars = 7 × 1500
    obs: 'cluster_id'

In [5]:
len(adata.var)

1500

In [6]:
adata.obs['cluster_id'].unique()

array(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7'], dtype=object)

In [7]:
adata.obs

,cluster_id
V1,V1
V2,V2
V3,V3
V4,V4
V5,V5
V6,V6
V7,V7


In [8]:
adata.var

""
MAGEC1
MAGEC2
GJB6
DSG1
DSC3
...
ITGB8
SH2D1A
SLC26A4
GDF15


In [9]:
adata.obs

,cluster_id
V1,V1
V2,V2
V3,V3
V4,V4
V5,V5
V6,V6
V7,V7


In [10]:
nmf_coeffs = pd.read_csv(data_path+'nmf150_H.csv', sep=',', index_col=0)
nmf_coeffs['cluster_id'] = nmf_coeffs.index.map(lambda x: 'V'+str(x))
nmf_coeffs = nmf_coeffs.set_index('cluster_id')
nmf_coeffs.head(2)

,CO39262-294984-2472,CO39262-294986-2469,CO39262-295209-3203,CO39262-295209-3204,CO39262-295210-3200,CO39262-295210-3207,CO39262-295241-3202,CO39262-295241-3205,CO39262-295241-3206,CO39262-295343-3201,...,CO39262-300786-2000,CO39262-300786-2001,CO39262-300786-2004,CO39262-300786-2009,CO39262-300937-2138,CO39262-300937-2146,CO39262-304465-1514,CO39262-304587-2971,CO39262-304587-2974,CO39262-305602-1007
cluster_id,,,,,,,,,,,,,,,,,,,,,
V1,2.946777e-02,0.004093,0.043211,0.119997,0.017096,0.029710,0.066921,0.000876,8.924199e-02,0.206239,...,7.659130e-02,0.160903,0.010032,0.000601,0.028044,0.022137,0.084895,0.019943,0.001255,0.200357
V2,2.220446e-16,0.095493,0.057197,0.041751,0.264143,0.263673,0.122844,0.073729,2.220446e-16,0.015092,...,2.220446e-16,0.004534,0.015386,0.007022,0.027242,0.120432,0.033406,0.000066,0.259836,0.004631


## Run with it

In [11]:
cluster_label='cluster_id'

In [12]:
sample_name='CO39262-297203-2231'

In [13]:
adata_ret = adata_to_nmf_factor_expression(adata=adata, 
                                           cluster_label = cluster_label, 
                                           nmf_coeffs = nmf_coeffs, 
                                           sample_name = sample_name, 
                                           add_density=True)

/gstore/home/tripaa18/miniconda3/envs/tangram-env/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [14]:
adata_ret

AnnData object with n_obs × n_vars = 7 × 1500
    obs: 'cluster_id', 'cluster_density'

In [17]:
nmf_coeffs[sample_name] / nmf_coeffs[sample_name].sum()

cluster_id
V1    3.170029e-01
V2    1.190991e-01
V3    6.060443e-16
V4    2.218394e-01
V5    1.882700e-01
V6    1.030036e-01
V7    5.078502e-02
Name: CO39262-297203-2231, dtype: float64

In [15]:
adata_ret.obs['cluster_density']

0    3.170029e-01
1    1.190991e-01
2    6.060443e-16
3    2.218394e-01
4    1.882700e-01
5    1.030036e-01
6    5.078502e-02
Name: cluster_density, dtype: float64

In [18]:
adata_ret.var

""
MAGEC1
MAGEC2
GJB6
DSG1
DSC3
...
ITGB8
SH2D1A
SLC26A4
GDF15


In [19]:
adata_ret.X

array([[0.8095371127128601, 0.2668927311897278, 0.03433814272284508, ...,
        1.8158457279205322, 2.356261730194092, 1.1201705932617188],
       [0.30051934719085693, 0.0918399915099144, 6.4512258514638344e-18,
        ..., 0.6470842361450195, 0.5741303563117981, 0.09122151881456375],
       [8.570383121855049e-16, 3.8185390320324754e-32,
        3.8185390320324754e-32, ..., 8.018663447100633e-16,
        3.8185390320324754e-32, 1.1263260738645265e-15],
       ...,
       [0.47570866346359253, 0.10212508589029312, 0.44500380754470825,
        ..., 0.7611825466156006, 0.8226573467254639, 0.5707228779792786],
       [0.17652665078639984, 0.03356831893324852, 0.6042942404747009,
        ..., 0.6698748469352722, 0.8362780213356018, 0.8895037770271301],
       [0.02817491628229618, 0.03572763875126839, 0.35892224311828613,
        ..., 0.1061231717467308, 0.11332416534423828,
        0.07599412649869919]], dtype=object)

# Scratch

In [134]:
try:
    unique_labels = adata.obs[cluster_label].unique()
except KeyError as e:
    raise ValueError("Provided label must belong to adata.obs.")

In [135]:
unique_labels

array(['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7'], dtype=object)

In [136]:
new_obs = pd.DataFrame({cluster_label: unique_labels})
adata_ret = sc.AnnData(obs=new_obs, var=adata.var, uns=adata.uns)
X_new = np.empty((len(unique_labels), adata.shape[1]))

/gstore/home/tripaa18/miniconda3/envs/tangram-env/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [137]:
X_new.shape

(7, 1500)

In [138]:
X_new[1].shape

(1500,)

In [139]:
np.multiply(adata[adata.obs[cluster_label] == l].X, nmf_coeffs[sample_name].loc[l]).shape

(1, 1500)

In [140]:
adata_ret

AnnData object with n_obs × n_vars = 7 × 1500
    obs: 'cluster_id'

In [141]:
X_new.shape

(7, 1500)

In [142]:
for index, l in enumerate(unique_labels):
    print(index)
    print(l)
    print(X_new[index].shape)
    print(np.multiply(adata[adata.obs[cluster_label] == l].X, nmf_coeffs[sample_name].loc[l]).shape)
# adata_ret.X = X_new

0
V1
(1500,)
(1, 1500)
1
V2
(1500,)
(1, 1500)
2
V3
(1500,)
(1, 1500)
3
V4
(1500,)
(1, 1500)
4
V5
(1500,)
(1, 1500)
5
V6
(1500,)
(1, 1500)
6
V7
(1500,)
(1, 1500)


In [143]:
adata

AnnData object with n_obs × n_vars = 7 × 1500
    obs: 'cluster_id'

In [144]:
adata[adata.obs['cluster_id'] == 'V1'].X

ArrayView([[ 6.970078  ,  2.2979343 ,  0.29564986, ..., 15.63435   ,
            20.287308  ,  9.644618  ]], dtype=float32)

In [145]:
nmf_coeffs[sample_name].loc[l]

0.0111911946253353

In [146]:
6.970078 * 0.0170963805549846

0.11916310598592596

In [147]:
np.multiply(adata['V1'].X, nmf_coeffs[sample_name].loc['V1'])

ArrayView([[0.1191631 , 0.03928636, 0.00505454, ..., 0.26729077,
            0.34683952, 0.16488805]], dtype=float32)

In [148]:
adata_ret.X

In [149]:
adata_ret[adata_ret.obs['cluster_id'] == 'V1'].X

In [150]:
adata_ret.obs["cluster_density"] = adata_ret.obs[cluster_label].map(
            lambda i: value_counts[i]
    )

NameError: name 'value_counts' is not defined

In [120]:
adata_ret

AnnData object with n_obs × n_vars = 7 × 1500
    obs: 'cluster_id', 'cluster_density'

In [123]:
adata_ret.obs[['cluster_density', 'cluster_id']]

,cluster_density,cluster_id
0,0.017096,V1
1,0.264143,V2
2,0.008988,V3
3,0.079838,V4
4,0.014583,V5
5,0.027974,V6
6,0.011191,V7


In [122]:
nmf_coeffs[sample_name]

cluster_id
V1    0.017096
V2    0.264143
V3    0.008988
V4    0.079838
V5    0.014583
V6    0.027974
V7    0.011191
Name: CO39262-295210-3200, dtype: float64